In [1]:
import numpy as np
lats = np.load("../data/didi/val.npy-lats.npy", allow_pickle=True)
lngs = np.load("../data/didi/val.npy-lngs.npy", allow_pickle=True)
labels = np.load("../data/didi/val.npy-labels.npy", allow_pickle=True)

In [2]:
labels[83418]

644

In [9]:
lats.shape

(363381,)

In [10]:
lngs.shape

(363381,)

In [12]:
"""
GPS CSV Format:
id;lng;lat
1;xxx;yyy
1;xxx;yyy
...
2;xxx;yyy
2;xxx;yyy
...
"""
length = len(lats)
f = open("../data/didi/val-gps.csv", "a+")
f.write(f"id;lng;lat\n")
for i in range(1, length + 1):
    lng = lngs[i - 1]
    lat = lats[i - 1]
    for x, y in zip(lng, lat):
        f.write(f"{i - 1};{x};{y}\n")
f.close()

In [1]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = gdf_edges.index
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

In [7]:
# Download by place name
# place = "Porto, Portugal"
place = "Chengdu"
# G = ox.graph_from_place(place, network_type='drive', which_result=2)
G = ox.graph_from_place(place, network_type='drive', which_result=1)
save_graph_shapefile_directional(G, filepath='chengdu')

In [1]:
import geopandas as gp

In [4]:
edge_shp = gp.GeoDataFrame.from_file("../../data/road_networks/porto/edges.shp")
# edge_shp = gp.GeoDataFrame.from_file("../data/chengdu/edges.shp")

In [5]:
node_shp = gp.GeoDataFrame.from_file("../../data/road_networks/porto/nodes.shp")
# node_shp = gp.GeoDataFrame.from_file("../data/chengdu/nodes.shp")

In [6]:
print(edge_shp)

            osmid oneway                   name      highway   length     ref  \
0       298888962   True  Avenida Vasco da Gama  residential   56.521    None   
1       298888969  False  Avenida Vasco da Gama  residential   15.502    None   
2       298888962   True  Avenida Vasco da Gama  residential   34.347    None   
3       298888971  False  Avenida Vasco da Gama  residential    6.977    None   
4       298888971  False  Avenida Vasco da Gama  residential    6.977    None   
...           ...    ...                    ...          ...      ...     ...   
182373   50729714  False                   None      primary  148.473  EN 204   
182374   35112471   True             Rua de Fez  residential  138.525    None   
182375   40769478   True             Rua de Fez  residential  126.632    None   
182376   95948590   True                   None      primary   18.903  EN 204   
182377  223117152  False                   None      primary  154.808  EN 204   

       bridge junction lane

In [7]:
print(node_shp)

               y         x       osmid highway   ref  \
0      41.228514 -8.617824  3028549663    None  None   
1      41.228809 -8.617318  3028549666    None  None   
2      41.228855 -8.617374  3028549667    None  None   
3      41.229016 -8.617014  3028549668    None  None   
4      41.229516 -8.616315  3028549671    None  None   
...          ...       ...         ...     ...   ...   
76560  41.067055 -8.578468   676855721    None  None   
76561  41.067116 -8.576226   676855724    None  None   
76562  41.354386 -8.466703   646709200    None  None   
76563  41.162013 -8.662884   314048477    None  None   
76564  41.352596 -8.467094   646709217    None  None   

                        geometry  
0      POINT (-8.61782 41.22851)  
1      POINT (-8.61732 41.22881)  
2      POINT (-8.61737 41.22885)  
3      POINT (-8.61701 41.22902)  
4      POINT (-8.61632 41.22952)  
...                          ...  
76560  POINT (-8.57847 41.06706)  
76561  POINT (-8.57623 41.06712)  
76562  POINT

In [8]:
len(edge_shp.osmid.unique())

57092

In [9]:
len(edge_shp.u.unique())

76501

In [11]:
len(edge_shp.v.unique())

76499

In [17]:
import numpy as np
len(np.unique(np.concatenate([edge_shp.u, edge_shp.v])))

76565

In [10]:
len(node_shp.osmid.unique())

76565

In [18]:
node_shp.columns

Index(['y', 'x', 'osmid', 'highway', 'ref', 'geometry'], dtype='object')

In [9]:
print(edge_shp.highway.unique())

['residential' 'primary' 'secondary' 'unclassified' 'tertiary'
 "['tertiary', 'residential']" 'primary_link' 'motorway_link'
 "['unclassified', 'residential']" 'tertiary_link' 'motorway'
 'living_street' 'secondary_link' 'trunk' 'trunk_link'
 "['primary', 'primary_link']" "['secondary', 'secondary_link']"
 "['tertiary', 'unclassified']" "['residential', 'living_street']"
 "['tertiary', 'unclassified', 'residential']"
 "['tertiary_link', 'tertiary']" "['unclassified', 'secondary']"
 "['primary', 'trunk']" "['trunk_link', 'trunk']"
 "['residential', 'secondary']" "['tertiary', 'secondary']" 'planned'
 "['tertiary', 'living_street']" 'escape' "['trunk', 'motorway_link']"
 "['unclassified', 'living_street']" "['motorway_link', 'motorway']"
 "['primary', 'residential']" "['residential', 'motorway_link']"
 "['tertiary_link', 'motorway_link']" "['trunk_link', 'primary_link']"
 "['primary', 'unclassified']" "['residential', 'planned']"
 "['secondary', 'motorway_link']" "['primary', 'tertiary']